<a href="https://colab.research.google.com/github/jakkapongz/hotel-reviews-sentiment-analysis/blob/develop/colab/hotel_reviews_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

## Install Libraries




In [ ]:
!pip uninstall torch torchtext -y
!pip uninstall torchaudio torchvision -y
!pip uninstall fastai torchmetrics -y
!pip -q install torch==1.5.0 torchtext==0.4.0 torchvision==0.6.0 -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip -q install pythainlp wordcloud thai2transformers

!pip list

!pip freeze

## Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)
# SET PATH TO DATA FOLDER
path= "/content/drive/Shareddrives/EGIT697_THEMATIC"

In [ ]:
!python --version

## Data Preparation

In [ ]:
import pandas as pd
import numpy as np

all_tsv_df = pd.read_csv(path + '/7500_good_longest_token_more_than_8_03082023.tsv', delimiter='\t')

all_tsv_df['label'].value_counts().plot.bar()

all_tsv_df

# all_df = pd.read_pickle(path + '/10000_good_bad_reviews_no_gap_02012023.pkl')

# all_df['label'].value_counts().plot.bar()

# all_df

# **Feature Extraction Method**


In [ ]:
import tensorflow as tf
import torch
import gc
import copy

# check GPU available?
torch.cuda.is_available()

## **Wangchanberta** (Monolingual Model)

In [ ]:
from transformers import (AutoTokenizer, AutoModel, pipeline, AutoModelForSequenceClassification)

# active GPU
# device = torch.device("cuda")
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

device = torch.device("cpu")

# model loading
wangchan_tokenizer = AutoTokenizer.from_pretrained(f'airesearch/wangchanberta-base-att-spm-uncased', output_hidden_states=True)
wangchan_model = AutoModel.from_pretrained(f'airesearch/wangchanberta-base-att-spm-uncased', output_hidden_states=True)
wangchan_model = wangchan_model.to(device)

In [ ]:
def adjust_encoded_input_wangchan(encoded_input):

  # delete first and last separator token and splits to 414 tokens
  input_ids_chunks = list(encoded_input['input_ids'][0][1:-1].split(414))
  attention_mask_chunks = list(encoded_input['attention_mask'][0][1:-1].split(414))

  for i in range(len(input_ids_chunks)):

    # add 5 to the first and 6 to last element tonsor padding len to 512 for transformer model 
    input_ids_chunks[i] = torch.cat([input_ids_chunks[i], torch.Tensor([6, 5]).long() ])
    # shifting 6 5
    input_ids_chunks[i] = torch.roll(input_ids_chunks[i], 1, 0)
    
    # padding len to 416 for transformer model
    pad_len = 416 - input_ids_chunks[i].shape[0]

    # 1 token represents padding <pad>
    input_ids_chunks[i] = torch.cat([input_ids_chunks[i], torch.Tensor([1] * pad_len).long()])

    if len(attention_mask_chunks[i]) == 414:
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], torch.Tensor([1] * 2).long()])
    else:
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], torch.Tensor([1] * 2).long()])
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], torch.Tensor([0] * (pad_len)).long()])

  input_ids = torch.stack(input_ids_chunks)
  attention_mask = torch.stack(attention_mask_chunks)

  input_dict = {
      'input_ids': input_ids.long(),
      'attention_mask': attention_mask.int()
  }

  return input_dict

adjust_encoded_input_wangchan method ที่ช่วยจัดการ split word tokens ที่ยาวเกินให้มีขนาด 416 ส่วนที่เหลือจะถูกทำการ padding ด้วย 1 ในฝั่งของ wangchanberta

In [ ]:
# EXAMPLE
text = "จากก้าวแรกที่ผ่านประตูทางเข้า ด้วยกลิ่นหอมของเครื่องหอมที่จัดไว้ของทางโรงแรมทำให้เกิดความรู้สึกของการต้อนรับที่อบอุ่นของเจ้าของโรงแรม ที่ไม่ได้เกิดจากบุคคล ผสมผสานกับการเอาใจใส่ของพนักงานที่ได้รับการฝึกอบรมมาอย่างดีทำให้รับรู้ได้ถึงคำว่าบริการ 5 ดาวในโรงแรมขนาดเล็ก ห้องพักที่เข้าพักเป็นห้องชั้นล่าง สำหรับตึก 2 ชั้นใกล้สระว่ายน้ำขนาดเล็ก ทำให้สะดวกในการเล่นน้ำในสระ ประกอบกับการออกแบบในสไตล์ Indochina ที่พบเห็นไม่บ่อยครั้งนัก ทำให้การพักผ่อนในครั้งนี้ นับเป็นการพักผ่อนที่สมบูรณ์แบบอีกครั้งหนึ่ง"

encoded_input = wangchan_tokenizer(text, return_tensors='pt').to(device)
encoded_input = adjust_encoded_input_wangchan(encoded_input)
encoded_input['input_ids'], encoded_input['input_ids'].size()

In [ ]:
def extract_last_four_with_wangchan(input_text, feature_extractor):

  encoded_input = wangchan_tokenizer(input_text, return_tensors='pt').to(device)
  encoded_input = adjust_encoded_input_wangchan(encoded_input)
  _, _, hidden_states = feature_extractor(**encoded_input)

  # use only last 4 layers 
  last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]

  # concat last 4 layers vectors then calculate mean between vectors
  cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
  cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()

  # if document only has 1 batch, no need to sum vector
  if cat_sentence_embedding.shape[0] != 3072:

    doc_embedding = torch.sum(cat_sentence_embedding, dim=0)
  else:
    doc_embedding = copy.copy(cat_sentence_embedding)
  
  return doc_embedding.cpu().detach().numpy().astype('float64')

We can use the outputs of WangchanBERTa (or any transformer-based models) as document vectors as an example by [BramVanroy](https://github.com/BramVanroy/bert-for-inference/blob/master/introduction-to-bert.ipynb).

![](https://github.com/BramVanroy/bert-for-inference/raw/ab7c57d6e7c79faf83ac0f9b6595c4b3d660c43c/img/bert-feature-extraction-contextualized-embeddings.png)

In [ ]:
text = "จากก้าวแรกที่ผ่านประตูทางเข้า ด้วยกลิ่นหอมของเครื่องหอมที่จัดไว้ของทางโรงแรมทำให้เกิดความรู้สึกของการต้อนรับที่อบอุ่นของเจ้าของโรงแรม ที่ไม่ได้เกิดจากบุคคล ผสมผสานกับการเอาใจใส่ของพนักงานที่ได้รับการฝึกอบรมมาอย่างดีทำให้รับรู้ได้ถึงคำว่าบริการ 5 ดาวในโรงแรมขนาดเล็ก ห้องพักที่เข้าพักเป็นห้องชั้นล่าง สำหรับตึก 2 ชั้นใกล้สระว่ายน้ำขนาดเล็ก ทำให้สะดวกในการเล่นน้ำในสระ ประกอบกับการออกแบบในสไตล์ Indochina ที่พบเห็นไม่บ่อยครั้งนัก ทำให้การพักผ่อนในครั้งนี้ นับเป็นการพักผ่อนที่สมบูรณ์แบบอีกครั้งหนึ่ง"

print(wangchan_model)

# t_2 = extract_last_four_with_wangchan(text, wangchan_model)
# t_2

In [ ]:
len(t_2)

## **Bert Model for Multilingual**

In [ ]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

device = torch.device("cpu")
# active GPU
# device = torch.device("cuda")
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', output_hidden_states=True)
bert_model = BertModel.from_pretrained("bert-base-multilingual-cased", output_hidden_states=True)
bert_model = bert_model.to(device)
# bert_class = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

In [ ]:
a = torch.Tensor([102, 101]).long()
c = torch.Tensor([1] * 2).long()
def adjust_encoded_input(encoded_input):

  # delete first and last separator token and splits to 510 tokens
  input_ids_chunks = list(encoded_input['input_ids'][0][1:-1].split(510))
  attention_mask_chunks = list(encoded_input['attention_mask'][0][1:-1].split(510))

  for i in range(len(input_ids_chunks)):

    # add 101 to the first and 102 to last element tonsor padding len to 512 for transformer model 
    input_ids_chunks[i] = torch.cat([input_ids_chunks[i], a ])
    # shifting 101 102
    input_ids_chunks[i] = torch.roll(input_ids_chunks[i], 1, 0)
    
    # padding len to 512 for transformer model
    pad_len = 512 - input_ids_chunks[i].shape[0]
    b = torch.Tensor([0] * pad_len).long()

    input_ids_chunks[i] = torch.cat([input_ids_chunks[i], b])

    if len(attention_mask_chunks[i]) == 510:
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], c])
    else:
      d = torch.Tensor([0] * (pad_len)).long()
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], c])
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], d])

  input_ids = torch.stack(input_ids_chunks)
  attention_mask = torch.stack(attention_mask_chunks)

  input_dict = {
      'input_ids': input_ids.long(),
      'attention_mask': attention_mask.int()
  }

  return input_dict

method ที่ช่วยจัดการ split word tokens ที่ยาวเกินให้มีขนาด 512 ส่วนที่เหลือจะถูกทำการ padding ด้วย 0 ในฝั่งของ bert

In [ ]:
text = "จากก้าวแรกที่ผ่านประตูทางเข้า ด้วยกลิ่นหอมของเครื่องหอมที่จัดไว้ของทางโรงแรมทำให้เกิดความรู้สึกของการต้อนรับที่อบอุ่นของเจ้าของโรงแรม ที่ไม่ได้เกิดจากบุคคล ผสมผสานกับการเอาใจใส่ของพนักงานที่ได้รับการฝึกอบรมมาอย่างดีทำให้รับรู้ได้ถึงคำว่าบริการ 5 ดาวในโรงแรมขนาดเล็ก ห้องพักที่เข้าพักเป็นห้องชั้นล่าง สำหรับตึก 2 ชั้นใกล้สระว่ายน้ำขนาดเล็ก ทำให้สะดวกในการเล่นน้ำในสระ ประกอบกับการออกแบบในสไตล์ Indochina ที่พบเห็นไม่บ่อยครั้งนัก ทำให้การพักผ่อนในครั้งนี้ นับเป็นการพักผ่อนที่สมบูรณ์แบบอีกครั้งหนึ่ง"

encoded_input = bert_tokenizer(text, return_tensors='pt').to(device)
encoded_input = adjust_encoded_input(encoded_input)
encoded_input['input_ids'], encoded_input['input_ids'].size()

In [ ]:
def extract_last_four_with_bert(input_text, feature_extractor):
  encoded_input = bert_tokenizer(input_text, return_tensors='pt').to(device)
  encoded_input = adjust_encoded_input_wangchan(encoded_input)
  # hidden_states = feature_extractor(**encoded_input)[0]
  _, _, hidden_states = feature_extractor(**encoded_input)

  # FOR MEAN CALCULATION BETWEEN TENSOR DIMENSION

  last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]

  cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
  cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()

  if cat_sentence_embedding.shape[0] != 3072:

    doc_embedding = torch.sum(cat_sentence_embedding, dim=0)
  else:
    doc_embedding = copy.copy(cat_sentence_embedding)

  return doc_embedding.cpu().detach().numpy().astype('float64')

In [ ]:
text = "จากก้าวแรกที่ผ่านประตูทางเข้า ด้วยกลิ่นหอมของเครื่องหอมที่จัดไว้ของทางโรงแรมทำให้เกิดความรู้สึกของการต้อนรับที่อบอุ่นของเจ้าของโรงแรม ที่ไม่ได้เกิดจากบุคคล ผสมผสานกับการเอาใจใส่ของพนักงานที่ได้รับการฝึกอบรมมาอย่างดีทำให้รับรู้ได้ถึงคำว่าบริการ 5 ดาวในโรงแรมขนาดเล็ก ห้องพักที่เข้าพักเป็นห้องชั้นล่าง สำหรับตึก 2 ชั้นใกล้สระว่ายน้ำขนาดเล็ก ทำให้สะดวกในการเล่นน้ำในสระ ประกอบกับการออกแบบในสไตล์ Indochina ที่พบเห็นไม่บ่อยครั้งนัก ทำให้การพักผ่อนในครั้งนี้ นับเป็นการพักผ่อนที่สมบูรณ์แบบอีกครั้งหนึ่ง"

t_2 = extract_last_four_with_bert(text, bert_model)
t_2

In [ ]:
len(t_2)

## **XLM-RoberTa Model for Multilingual**

In [ ]:
from transformers import AutoTokenizer, XLMRobertaModel

device = torch.device("cpu")

xlmr_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", output_hidden_states=True)
xlmr_model = XLMRobertaModel.from_pretrained("xlm-roberta-base", output_hidden_states=True)
xlmr_model = xlmr_model.to(device)

In [ ]:
a = torch.Tensor([102, 101]).long()
c = torch.Tensor([1] * 2).long()
def adjust_encoded_input_xlmr(encoded_input):

  # delete first and last separator token and splits to 510 tokens
  input_ids_chunks = list(encoded_input['input_ids'][0][1:-1].split(510))
  attention_mask_chunks = list(encoded_input['attention_mask'][0][1:-1].split(510))

  for i in range(len(input_ids_chunks)):

    # add 101 to the first and 102 to last element tonsor padding len to 512 for transformer model 
    input_ids_chunks[i] = torch.cat([input_ids_chunks[i], a ])
    # shifting 101 102
    input_ids_chunks[i] = torch.roll(input_ids_chunks[i], 1, 0)
    
    # padding len to 512 for transformer model
    pad_len = 512 - input_ids_chunks[i].shape[0]
    b = torch.Tensor([0] * pad_len).long()

    input_ids_chunks[i] = torch.cat([input_ids_chunks[i], b])

    if len(attention_mask_chunks[i]) == 510:
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], c])
    else:
      d = torch.Tensor([0] * (pad_len)).long()
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], c])
      attention_mask_chunks[i] = torch.cat([attention_mask_chunks[i], d])

  input_ids = torch.stack(input_ids_chunks)
  attention_mask = torch.stack(attention_mask_chunks)

  input_dict = {
      'input_ids': input_ids.long(),
      'attention_mask': attention_mask.int()
  }

  return input_dict

In [ ]:
# EXAMPLE
text = "จากก้าวแรกที่ผ่านประตูทางเข้า ด้วยกลิ่นหอมของเครื่องหอมที่จัดไว้ของทางโรงแรมทำให้เกิดความรู้สึกของการต้อนรับที่อบอุ่นของเจ้าของโรงแรม ที่ไม่ได้เกิดจากบุคคล ผสมผสานกับการเอาใจใส่ของพนักงานที่ได้รับการฝึกอบรมมาอย่างดีทำให้รับรู้ได้ถึงคำว่าบริการ 5 ดาวในโรงแรมขนาดเล็ก ห้องพักที่เข้าพักเป็นห้องชั้นล่าง สำหรับตึก 2 ชั้นใกล้สระว่ายน้ำขนาดเล็ก ทำให้สะดวกในการเล่นน้ำในสระ ประกอบกับการออกแบบในสไตล์ Indochina ที่พบเห็นไม่บ่อยครั้งนัก ทำให้การพักผ่อนในครั้งนี้ นับเป็นการพักผ่อนที่สมบูรณ์แบบอีกครั้งหนึ่ง"

encoded_input = xlmr_tokenizer(text, return_tensors='pt').to(device)
encoded_input = adjust_encoded_input_xlmr(encoded_input)
encoded_input['input_ids'], encoded_input['input_ids'].size()

In [ ]:
def extract_last_four_with_xlmr(input_text, feature_extractor):
  encoded_input = xlmr_tokenizer(input_text, return_tensors='pt').to(device)
  encoded_input = adjust_encoded_input_wangchan(encoded_input)
  # hidden_states = feature_extractor(**encoded_input)[0]
  _, _, hidden_states = feature_extractor(**encoded_input)

  # FOR MEAN CALCULATION BETWEEN TENSOR DIMENSION

  last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]

  cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
  cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()

  if cat_sentence_embedding.shape[0] != 3072:

    doc_embedding = torch.sum(cat_sentence_embedding, dim=0)
  else:
    doc_embedding = copy.copy(cat_sentence_embedding)

  return doc_embedding.cpu().detach().numpy().astype('float64')

In [ ]:
text = "จากก้าวแรกที่ผ่านประตูทางเข้า ด้วยกลิ่นหอมของเครื่องหอมที่จัดไว้ของทางโรงแรมทำให้เกิดความรู้สึกของการต้อนรับที่อบอุ่นของเจ้าของโรงแรม ที่ไม่ได้เกิดจากบุคคล ผสมผสานกับการเอาใจใส่ของพนักงานที่ได้รับการฝึกอบรมมาอย่างดีทำให้รับรู้ได้ถึงคำว่าบริการ 5 ดาวในโรงแรมขนาดเล็ก ห้องพักที่เข้าพักเป็นห้องชั้นล่าง สำหรับตึก 2 ชั้นใกล้สระว่ายน้ำขนาดเล็ก ทำให้สะดวกในการเล่นน้ำในสระ ประกอบกับการออกแบบในสไตล์ Indochina ที่พบเห็นไม่บ่อยครั้งนัก ทำให้การพักผ่อนในครั้งนี้ นับเป็นการพักผ่อนที่สมบูรณ์แบบอีกครั้งหนึ่ง"

t_2 = extract_last_four_with_xlmr(text, xlmr_model)
t_2

In [ ]:
len(t_2)

## Extract last four from data

In [ ]:
wangchan_vectors = []
bert_vectors = []
xlmr_vectors = []

for idx, row in all_tsv_df.iterrows():
  text = row['review_text']
  wangchan_vector = extract_last_four_with_wangchan(text, wangchan_model)
  bert_vector = extract_last_four_with_bert(text, bert_model)
  xlmr_vector = extract_last_four_with_xlmr(text, xlmr_model)
  wangchan_vectors.append(wangchan_vector)
  bert_vectors.append(bert_vector)
  xlmr_vectors.append(xlmr_vector)
  print(idx)

all_tsv_df['content_bert_vector'] = bert_vectors
all_tsv_df['content_wangchan_vector'] = wangchan_vectors
all_tsv_df['content_xlmr_vector'] = xlmr_vectors

all_tsv_df.to_pickle(path + "/7500_good_longest_token_more_than_8_03082023.tsv.pkl")

In [ ]:
pos_df = pd.read_pickle(path + '/5000_good_reviews_no_gap_02012023.pkl')

pos_df['label'].value_counts().plot.bar()

pos_df


In [ ]:
neg_df = pd.read_pickle(path + '/5000_bad_reviews_no_gap_02012023.pkl')

neg_df['label'].value_counts().plot.bar()

neg_df

In [ ]:
combined_df = neg_df.append(pos_df, ignore_index=True)

# # # combined_df = pos_df.append(neg_df, ignore_index=True)

combined_df.rename(columns = {'review_token':'review_tokens'}, inplace = True)
combined_df

combined_df.to_pickle(path + "/10000_good_bad_reviews_no_gap_02012023.pkl")

In [ ]:
# SAMPLE GOOD TEXT WANGCHAN
text = all_df.iloc[5]['review_text']

print(text + '\n')

encoded_input = wangchan_tokenizer(text, return_tensors='pt').to(device)
encoded_input = adjust_encoded_input_wangchan(encoded_input)
encoded_input['input_ids'], encoded_input['input_ids'].size()

In [ ]:
# SAMPLE BAD TEXT WANGCHAN
text = all_df.iloc[5002]['review_text']

print(text + '\n')

encoded_input = wangchan_tokenizer(text, return_tensors='pt').to(device)
encoded_input = adjust_encoded_input_wangchan(encoded_input)
encoded_input['input_ids'], encoded_input['input_ids'].size()

## PyThai NLP

### Pre-Processing

####Light Clean

In [ ]:
import string
import re
from pythainlp import word_tokenize
from pythainlp.corpus.common import thai_stopwords
from pythainlp import sent_tokenize, word_tokenize

thai_stopwords = list(thai_stopwords())

def text_process(text):

    final = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "ๆ", "ฯ"))
    final = final.translate(str.maketrans('','', string.punctuation))

    final = word_tokenize(final)
    final = " ".join(word for word in final)
    return final

all_df['review_tokens'] = all_df['review_text'].apply(text_process)
# all_df.head()

X = all_df[['review_tokens']]
y = all_df['label']

####Deep Clean

In [ ]:
import string
import re
from pythainlp import word_tokenize
from pythainlp.corpus.common import thai_stopwords
from pythainlp import sent_tokenize, word_tokenize
from thai2transformers import preprocess

thai_stopwords = list(thai_stopwords())

def text_process(text):

    t = preprocess.fix_html(text)
    t = preprocess.rm_brackets(t)
    t = preprocess.replace_newlines(t)
    t = preprocess.rm_useless_spaces(t)
    t = preprocess.replace_spaces(t)
    t = preprocess.replace_rep_after(t)

    tokens = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "ๆ", "ฯ"))
    tokens = tokens.translate(str.maketrans('','', string.punctuation))

    tokens = word_tokenize(tokens)
    tokens = preprocess.ungroup_emoji(tokens)
    tokens = preprocess.replace_wrep_post(tokens)

    final = " ".join(word for word in tokens)

    return final

all_df['deep_clean_review_tokens'] = all_df['review_text'].apply(text_process)
all_df.head()

deep_clean_X = all_df[['deep_clean_review_tokens']]
deep_clean_y = all_df['label']

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(analyzer=lambda x:x.split(' '))

tfidf_vec = tfidf_vectorizer.fit_transform(X['review_tokens'])
tfidf_array = np.array(tfidf_vec.todense())

content_tfidf = []

for vec in tfidf_array:
  content_tfidf.append(vec)

all_df['content_tfidf_vector'] = content_tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(analyzer=lambda x:x.split(' '))

tfidf_vec = tfidf_vectorizer.fit_transform(deep_clean_X['deep_clean_review_tokens'])
tfidf_array = np.array(tfidf_vec.todense())

content_tfidf = []

for vec in tfidf_array:
  content_tfidf.append(vec)

all_df['content_deep_clean_tfidf_vector'] = content_tfidf

### Bag Of Word

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(analyzer=lambda x:x.split(' '))

bow_vec = count_vectorizer.fit_transform(X['review_tokens'])
bow_array = np.array(bow_vec.todense())

content_bow = []

for vec in bow_array:
  content_bow.append(vec)

all_df['content_bow_vector'] = content_bow


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(analyzer=lambda x:x.split(' '))

bow_vec = count_vectorizer.fit_transform(deep_clean_X['deep_clean_review_tokens'])
bow_array = np.array(bow_vec.todense())

content_bow = []

for vec in bow_array:
  content_bow.append(vec)

all_df['content_deep_clean_bow_vector'] = content_bow

In [ ]:
all_df.to_pickle(path + "/10000_good_bad_reviews_no_gap_02012023_completed.pkl")

# Word Cloud

## Good words

In [ ]:
import string
import re
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS
from pythainlp.corpus.common import thai_stopwords

thai_stopwords = list(thai_stopwords())


df_pos = all_df[all_df['label'] == 'GOOD']
pos_word_all = " ".join(text for text in df_pos['deep_clean_review_tokens'])
# print(pos_word_all)
reg = r"[ก-๙a-zA-Z']+"
fp =  path + '/THSarabunNew.ttf'
wordcloud = WordCloud(stopwords=thai_stopwords, background_color = 'white', max_words=2000, height = 2000, width=4000, font_path=fp, regexp=reg).generate(pos_word_all)
plt.figure(figsize = (16,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## Bad words

In [ ]:
import string
import re
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS
from pythainlp.corpus.common import thai_stopwords

thai_stopwords = list(thai_stopwords())


df_neg = all_df[all_df['label'] == 'BAD']
neg_word_all = " ".join(text for text in df_neg['deep_clean_review_tokens'])
# print(pos_word_all)
reg = r"[ก-๙a-zA-Z']+"
fp =  path + '/THSarabunNew.ttf'
wordcloud = WordCloud(stopwords=thai_stopwords, background_color = 'white', max_words=2000, height = 2000, width=4000, font_path=fp, regexp=reg).generate(neg_word_all)
plt.figure(figsize = (16,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# Predictive Model Running

In [ ]:
all_df.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support , classification_report
import matplotlib.pyplot as plt

In [ ]:
ks_selected_df = all_df

# train test spilt 70/30 ratio
ks_df_train, ks_df_test, ks_df_y_train, ks_df_y_test = train_test_split(ks_selected_df, list(ks_selected_df['label']), test_size=0.3, random_state=0)

task = {
    "ks_df": {
        'data': ks_df_train,
        'col': 'content_bert_vector',
        'language_model' : 'BERT (multilingual)'
    },
    "kh_df": {
        'data': ks_df_train,
        'col': 'content_wangchan_vector',
        'language_model' : 'WangchanBERTa (monolingual)'
    },
    "xlmr_df": {
        'data': ks_df_train,
        'col': 'content_xlmr_vector',
        'language_model' : 'XML-RoBERTa (multilingual)'
    },
    "tfidf": {
        'data': ks_df_train,
        'col': 'content_tfidf_vector',
        'language_model' : 'TF-IDF'
    },
    "bow": {
        'data': ks_df_train,
        'col': 'content_bow_vector',
        'language_model' : 'Bag Of Word'
    },
    "deep_clean_tfidf": {
        'data': ks_df_train,
        'col': 'content_deep_clean_tfidf_vector',
        'language_model' : 'TF-IDF (Deep clean)'
    },
    "bow": {
        'data': ks_df_train,
        'col': 'content_deep_clean_bow_vector',
        'language_model' : 'Bag Of Word (Deep clean)'
    }
    
}

In [ ]:
ks_selected_df['label'].value_counts().plot.bar()

In [ ]:
is_manual = True

# loop data df
for i in task:

  col = task[i]['col']
  X = list(task[i]['data'][col])
  y = list(task[i]['data']['label'])

  X_test = list(ks_df_test[col])
  y_test = list(ks_df_test['label'])

  max_num_iter = 1000

  if is_manual:

    logreg_model = LogisticRegression(max_iter=max_num_iter, random_state=0,multi_class='multinomial')
    logreg_model.fit(X, y)
    y_pred = logreg_model.predict(X_test)
    print(task[i]['language_model'])
    print(classification_report(y_pred, y_test, digits = 4))

    print('\n')